In [17]:
import numpy as np
import random

# Constants
c = 3.0e8  # Speed of light in m/s
k_e = 8.9875517873681764e9  # Coulomb's constant in N m²/C²
fusion_energy_threshold = 1e12  # Threshold for fusion in joules
collision_threshold = 1e-10  # Distance threshold for proximity-based collision detection

# Particle Class
class Particle:
    def __init__(self, position, velocity, mass, charge, p_type):
        self.position = np.array(position)
        self.velocity = np.array(velocity)
        self.mass = mass
        self.charge = charge
        self.type = p_type
        self.rest_energy = self.calculate_rest_energy()

    def calculate_rest_energy(self):
        return self.mass * c**2

    def calculate_momentum(self):
        return self.mass * np.linalg.norm(self.velocity)

    def total_energy(self):
        momentum = self.calculate_momentum()
        return np.sqrt((self.rest_energy)**2 + (momentum * c)**2)

# Collider Class
class Collider:
    def __init__(self, particles):
        self.particles = particles

    def detect_collisions(self):
        """Detect collisions based on proximity between particles."""
        for i, p1 in enumerate(self.particles):
            for j, p2 in enumerate(self.particles[i+1:], start=i+1):
                distance = np.linalg.norm(p1.position - p2.position)
                if distance < collision_threshold:
                    self.handle_collision(p1, p2)

    def handle_collision(self, p1, p2):
        """Handle a collision between two particles, checking for fusion or transformation."""
        total_initial_momentum = p1.mass * p1.velocity + p2.mass * p2.velocity
        total_initial_energy = p1.total_energy() + p2.total_energy()
        total_charge = p1.charge + p2.charge  # Ensure charge conservation

        # Check if fusion occurs
        if total_initial_energy > fusion_energy_threshold:
            self.perform_fusion(p1, p2, total_initial_momentum, total_charge)
        else:
            # Apply momentum conservation and check for stochastic transformations
            self.apply_momentum_conservation(p1, p2)
            self.stochastic_transformation(p1, p2)

    def apply_momentum_conservation(self, p1, p2):
        """Apply conservation of momentum and energy during elastic collision."""
        if p1.mass != p2.mass:
            v1_final = ((p1.mass - p2.mass) / (p1.mass + p2.mass)) * p1.velocity + \
                       (2 * p2.mass / (p1.mass + p2.mass)) * p2.velocity
            v2_final = ((p2.mass - p1.mass) / (p1.mass + p2.mass)) * p2.velocity + \
                       (2 * p1.mass / (p1.mass + p2.mass)) * p1.velocity
            p1.velocity = v1_final
            p2.velocity = v2_final
        else:
            # Swap velocities if masses are equal
            p1.velocity, p2.velocity = p2.velocity, p1.velocity

    def stochastic_transformation(self, p1, p2):
        """Randomly transform particles post-collision based on a probability."""
        transformation_probability = 0.3  # 30% chance of transformation
        if np.random.rand() < transformation_probability:
            p1.type = "DecayProduct1"
            p2.type = "DecayProduct2"
            p1.mass *= 0.5
            p2.mass *= 0.5
            print(f"Particles transformed into {p1.type} and {p2.type}.")

    def perform_fusion(self, p1, p2, total_initial_momentum, total_charge):
        """Fuse two particles into a single particle with combined mass and charge."""
        combined_mass = p1.mass + p2.mass
        additional_mass = ((p1.total_energy() + p2.total_energy()) - (combined_mass * c**2)) / c**2
        new_mass = combined_mass + additional_mass
        new_velocity = total_initial_momentum / new_mass
        fusion_particle = Particle(
            position=p1.position,
            velocity=new_velocity,
            mass=new_mass,
            charge=total_charge,
            p_type="FusionParticle"
        )
        self.particles.append(fusion_particle)
        print(f"Fusion created a new particle with mass {new_mass:.2e} kg and charge {total_charge}.")

    def apply_forces(self):
        """Apply electric forces between all particle pairs."""
        time_step = 0.01
        for i, p1 in enumerate(self.particles):
            for p2 in self.particles[i+1:]:
                force = self.calculate_electric_force(p1, p2)
                p1_acceleration = force / p1.mass
                p2_acceleration = -force / p2.mass
                p1.velocity += p1_acceleration * time_step
                p2.velocity += p2_acceleration * time_step

    def calculate_electric_force(self, p1, p2):
        """Calculate the electric force between two charged particles using Coulomb's law."""
        distance_vector = p2.position - p1.position
        distance = np.linalg.norm(distance_vector)
        if distance == 0:
            return np.zeros(3)  # Avoid division by zero
        force_magnitude = k_e * (p1.charge * p2.charge) / distance**2
        force_direction = distance_vector / distance
        return force_magnitude * force_direction

# Main Function
def main():
    # Create random particles for testing
    num_particles = 5
    particle_types = ["Proton", "Electron", "Muon", "Neutron"]
    particles = []
    for _ in range(num_particles):
        position = [random.uniform(-1e-9, 1e-9) for _ in range(3)]
        velocity = [random.uniform(-1e5, 1e5) for _ in range(3)]
        mass = random.uniform(1e-27, 1e-25)
        charge = random.choice([-1, 0, 1]) * 1.6e-19
        p_type = random.choice(particle_types)
        particles.append(Particle(position, velocity, mass, charge, p_type))

    # Initialize the Collider
    collider = Collider(particles)

    # Detect collisions
    print("Detecting collisions...")
    collider.detect_collisions()

    # Apply forces
    print("Applying forces...")
    collider.apply_forces()

    # Print particle states
    print("\nFinal particle states:")
    for i, particle in enumerate(collider.particles):
        print(f"Particle {i + 1}:")
        print(f" - Type: {particle.type}")
        print(f" - Position: {particle.position}")
        print(f" - Velocity: {particle.velocity}")
        print(f" - Mass: {particle.mass:.2e} kg")
        print(f" - Charge: {particle.charge:.2e} C")
        print(f" - Total Energy: {particle.total_energy():.2e} J\n")

if __name__ == "__main__":
    main()

Detecting collisions...
Applying forces...

Final particle states:
Particle 1:
 - Type: Neutron
 - Position: [-3.05719445e-10 -9.34996647e-10 -7.08896995e-10]
 - Velocity: [ 3.97028071e+13 -3.69315192e+13 -6.91158598e+12]
 - Mass: 7.58e-26 kg
 - Charge: 1.60e-19 C
 - Total Energy: 1.24e-03 J

Particle 2:
 - Type: Electron
 - Position: [-9.10178951e-10 -4.46087473e-10 -6.19232509e-10]
 - Velocity: [-2.86578585e+13  2.86760914e+13  7.29955853e+12]
 - Mass: 9.38e-26 kg
 - Charge: -1.60e-19 C
 - Total Energy: 1.16e-03 J

Particle 3:
 - Type: Muon
 - Position: [ 9.58175573e-10 -7.47063659e-10  8.59202481e-10]
 - Velocity: [ 7.12357284e+13 -1.07438942e+14  2.32755753e+13]
 - Mass: 1.15e-26 kg
 - Charge: 1.60e-19 C
 - Total Energy: 4.51e-04 J

Particle 4:
 - Type: Neutron
 - Position: [-9.81427182e-10  4.05348318e-10 -6.79695383e-10]
 - Velocity: [94856.56944582 10435.39738472 60111.72698929]
 - Mass: 6.55e-26 kg
 - Charge: 0.00e+00 C
 - Total Energy: 5.89e-09 J

Particle 5:
 - Type: Electron